In [7]:
f = 'text.txt'

In [16]:
file = open(f,'r')
text = ''
for line in file.readlines():
    text+=str(line)
    text+=" "
file.close()

In [17]:
print(text)

NASA to explore how 3-year Mars mission could affect humans
 WASHINGTON: As a round-trip mission to Mars could last three years, NASA is calling for more research to understand how long-term experiences in space could affect the human body.
 Typical missions to the International Space Station (ISS) last just six months. "To draw any conclusions about the cumulative effects of exposure to space, we need to observe more astronauts spending larger amounts of time in the space environment," said John Charles of the Human Research Programme at NASA's Johnson Space Center.
 Combined with ongoing NASA studies, the new research could enable safer and more effective travel to destinations beyond low-Earth orbit.The US space agency hopes that such research will help it understand, prevent, diagnose, treat, mitigate and cure the potential health effects of prolonged spaceflight. "Scientists can use the information to predict physical and behavioural health trends," Charles said.
 Proposals are du

In [19]:
import nltk
from nltk import word_tokenize
import string

text1 = word_tokenize(text)    #tokenize by word
case_insensitive_text = word_tokenize(text.lower()) #lowercase

In [22]:
#Segmentating Sentences
sentences = []
tokenized_sentences = []
sentence = " "
for word in text1:
    if word != '.':
        sentence+=str(word)+" "
    else:
        sentences.append(sentence.strip())
        tokenized_sentences.append(word_tokenize(sentence.lower().strip()))
        sentence = " "

In [24]:
sentences

['NASA to explore how 3-year Mars mission could affect humans WASHINGTON : As a round-trip mission to Mars could last three years , NASA is calling for more research to understand how long-term experiences in space could affect the human body',
 'Typical missions to the International Space Station ( ISS ) last just six months',
 "`` To draw any conclusions about the cumulative effects of exposure to space , we need to observe more astronauts spending larger amounts of time in the space environment , '' said John Charles of the Human Research Programme at NASA 's Johnson Space Center",
 'Combined with ongoing NASA studies , the new research could enable safer and more effective travel to destinations beyond low-Earth orbit.The US space agency hopes that such research will help it understand , prevent , diagnose , treat , mitigate and cure the potential health effects of prolonged spaceflight',
 "`` Scientists can use the information to predict physical and behavioural health trends , ''

In [25]:
tokenized_sentences

[['nasa',
  'to',
  'explore',
  'how',
  '3-year',
  'mars',
  'mission',
  'could',
  'affect',
  'humans',
  'washington',
  ':',
  'as',
  'a',
  'round-trip',
  'mission',
  'to',
  'mars',
  'could',
  'last',
  'three',
  'years',
  ',',
  'nasa',
  'is',
  'calling',
  'for',
  'more',
  'research',
  'to',
  'understand',
  'how',
  'long-term',
  'experiences',
  'in',
  'space',
  'could',
  'affect',
  'the',
  'human',
  'body'],
 ['typical',
  'missions',
  'to',
  'the',
  'international',
  'space',
  'station',
  '(',
  'iss',
  ')',
  'last',
  'just',
  'six',
  'months'],
 ['``',
  'to',
  'draw',
  'any',
  'conclusions',
  'about',
  'the',
  'cumulative',
  'effects',
  'of',
  'exposure',
  'to',
  'space',
  ',',
  'we',
  'need',
  'to',
  'observe',
  'more',
  'astronauts',
  'spending',
  'larger',
  'amounts',
  'of',
  'time',
  'in',
  'the',
  'space',
  'environment',
  ',',
  "''",
  'said',
  'john',
  'charles',
  'of',
  'the',
  'human',
  'resear

In [27]:
#Lemmatization (lemmas are actual words)

from nltk.stem import WordNetLemmatizer
def lemmatize(POS_tagged_text):
    
    wordnet_lemmatizer = WordNetLemmatizer()
    adjective_tags = ['JJ','JJR','JJS']
    lemmatized_text = []
    
    for word in POS_tagged_text:
        if word[1] in adjective_tags:
            lemmatized_text.append(str(wordnet_lemmatizer.lemmatize(word[0],pos="a")))
        else:
            lemmatized_text.append(str(wordnet_lemmatizer.lemmatize(word[0]))) #default POS = noun
    
    return lemmatized_text
#pos_tag attaches a part of speech tag to each word NN, a noun JJ, adj...

POS_tagged_text = nltk.pos_tag(case_insensitive_text)
lemmatized_text = lemmatize(POS_tagged_text)

In [28]:
Processed_text = nltk.pos_tag(lemmatized_text)

In [29]:
def generate_stopwords(POS_tagged_text):
    stopwords = []
    
    wanted_POS = ['NN','NNS','NNP','NNPS','JJ','JJR','JJS','FW'] #may be add VBG too
    
    for word in POS_tagged_text:
        if word[1] not in wanted_POS:
            stopwords.append(word[0])
            
    punctuations = list(str(string.punctuation))
    stopwords = stopwords + punctuations
    
    stopword_file = open("longstopwords.txt", "r")
    #Source = https://www.ranks.nl/stopwords

    for line in stopword_file.readlines():
        stopwords.append(str(line.strip()))

    return set(stopwords)

stopwords = generate_stopwords(Processed_text)

In [30]:
#Lemmatizing words in the sentences, and removing stopwords.
processed_sentences = []

for sentence in tokenized_sentences:
    processed_sentence = []
    
    POS_tagged_sentence = nltk.pos_tag(sentence)
    lemmatized_sentence = lemmatize(POS_tagged_sentence)

    for word in lemmatized_sentence:
        if word not in stopwords:
            processed_sentence.append(word)
    processed_sentences.append(processed_sentence)

In [42]:
#Build Graph
#TextRank - graph based model
#weighted_edge[i][j] contains the weight of the connecting edge between the sentence vertex represented by sentence of index i and the sentence vertex represented by sentence of index j
#The weight of the connection is the value of the similary between the connected vertices.
#The value of the weighted_edge[i][j] is the determined by the similarity function.
#The similarity function is: (No. of overlapping words in sentence i and j)/(log(len(sentence_i)) + log(len(sentence_j))
import numpy as np
import math
from __future__ import division

sentence_len = len(processed_sentences)
weighted_edge = np.zeros((sentence_len,sentence_len),dtype=np.float32)

score = np.zeros((sentence_len),dtype=np.float32)

for i in range(0,sentence_len):
    score[i]=1
    for j in range(0,sentence_len):
        if j==i:
            weighted_edge[i][j]=0
        else:
            for word in processed_sentences[i]:
                if word in processed_sentences[j]:
                    weighted_edge[i][j] += processed_sentences[j].count(word)
                    #weight edge will consist of the count of all common words b/w sentences/nodes of graph
            if weighted_edge[i][j]!=0:
                len_i = len(processed_sentences[i])
                len_j = len(processed_sentences[j])
                weighted_edge[i][j] = weighted_edge[i][j]/(math.log(len_i)+math.log(len_j))
                #was in algo dividing by total no. of connecting edges

In [43]:
#inout[i] will contain the sum of all the undirected connections\edges associated withe the vertex represented by i.
inout = np.zeros((sentence_len),dtype=np.float32)

for i in range(0,sentence_len):
    for j in range(0,sentence_len):
        inout[i]+=weighted_edge[i][j]

In [45]:
#Scoring using pagerank algorithm
#The formula used for scoring a vertex represented by i is:
#score[i] = (1-d) + d x [ Summation(j) ( (weighted_edge[i][j]/inout[j]) x score[j] ) ] where j belongs to the list of vertieces that has a connection with i.

MAX_ITERATIONS = 50
d=0.85  #damping factor
threshold = 0.0001 #convergence threshold

for iter in range(0,MAX_ITERATIONS):
    prev_score = np.copy(score)
    
    for i in range(0,sentence_len):
        
        summation = 0
        for j in range(0,sentence_len):
            if weighted_edge[i][j] != 0:
                summation += (weighted_edge[i][j]/inout[j])*score[j]
        score[i] = (1-d) + d*(summation) #pagerank algo
        #probability any person continues is d...summation is the sum of (pagerank of the vertex connected / outbounds to it)
    if np.sum(np.fabs(prev_score-score)) <= threshold: #convergence condition
        print ("Converging at iteration "+str(iter)+"....")
        break


Converging at iteration 25....


In [46]:
#Displays each sentence with score
i=0
for sentence in sentences:
    print("Sentence:\n\n"+str(sentence)+"\nScore: "+str(score[i])+"\n\n")
    i+=1

Sentence:

NASA to explore how 3-year Mars mission could affect humans WASHINGTON : As a round-trip mission to Mars could last three years , NASA is calling for more research to understand how long-term experiences in space could affect the human body
Score: 1.54634


Sentence:

Typical missions to the International Space Station ( ISS ) last just six months
Score: 0.990184


Sentence:

`` To draw any conclusions about the cumulative effects of exposure to space , we need to observe more astronauts spending larger amounts of time in the space environment , '' said John Charles of the Human Research Programme at NASA 's Johnson Space Center
Score: 1.6698


Sentence:

Combined with ongoing NASA studies , the new research could enable safer and more effective travel to destinations beyond low-Earth orbit.The US space agency hopes that such research will help it understand , prevent , diagnose , treat , mitigate and cure the potential health effects of prolonged spaceflight
Score: 0.824699

In [50]:
#Summary
#Sentences are then ranked in accordance to their corresponding scores
#The summary is then generated by presenting the sentences (whose indices were chosen) in a chronological order.
#selection of the first statement (if the summary_size is computed to be more than 1) because the first sentence can usually serve as an introduction, and provide some context to the topic.
Reduce_to_percent = 20
summary_size = int(((Reduce_to_percent)/100)*len(sentences))

if summary_size == 0:
    summary_size = 1

sorted_sentence_score_indices = np.flip(np.argsort(score),0)

indices_for_summary_results = sorted_sentence_score_indices[0:summary_size]

summary = "\n"

current_size = 0

if 0 not in indices_for_summary_results and summary_size!=1:
    summary+=sentences[0]
    summary+=".\n"
    current_size+=1


for i in range(0,len(sentences)):
    if i in indices_for_summary_results:
        summary+=sentences[i]
        summary+=".\n\n"
        current_size += 1
    if current_size == summary_size:
        break

print("\nSUMMARY: ")
print(summary)


SUMMARY: 

NASA to explore how 3-year Mars mission could affect humans WASHINGTON : As a round-trip mission to Mars could last three years , NASA is calling for more research to understand how long-term experiences in space could affect the human body.
`` To draw any conclusions about the cumulative effects of exposure to space , we need to observe more astronauts spending larger amounts of time in the space environment , '' said John Charles of the Human Research Programme at NASA 's Johnson Space Center.




In [49]:
print(text)  #Text taken at the beginning

NASA to explore how 3-year Mars mission could affect humans
 WASHINGTON: As a round-trip mission to Mars could last three years, NASA is calling for more research to understand how long-term experiences in space could affect the human body.
 Typical missions to the International Space Station (ISS) last just six months. "To draw any conclusions about the cumulative effects of exposure to space, we need to observe more astronauts spending larger amounts of time in the space environment," said John Charles of the Human Research Programme at NASA's Johnson Space Center.
 Combined with ongoing NASA studies, the new research could enable safer and more effective travel to destinations beyond low-Earth orbit.The US space agency hopes that such research will help it understand, prevent, diagnose, treat, mitigate and cure the potential health effects of prolonged spaceflight. "Scientists can use the information to predict physical and behavioural health trends," Charles said.
 Proposals are du